In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score

In [5]:
#экспорт файла1
cascresto=pd.read_csv('D:\casc-resto.csv', sep=';',decimal=",")
cascresto.head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.0
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0


In [14]:
cascresto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881608 entries, 0 to 882221
Data columns (total 9 columns):
CustomerID              881608 non-null int64
Restaurant              881608 non-null int64
RKDate                  881608 non-null datetime64[ns]
RegionName              881608 non-null object
BrandsNames             881608 non-null object
DishCategoryName        881608 non-null object
Quantity                881608 non-null int64
SummBasic               881608 non-null float64
SummAfterPointsUsage    881608 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 67.3+ MB


In [ ]:
#преобразование данных к типу дата
cascresto['RKDate'] = pd.to_datetime(cascresto['RKDate'])
cascresto.info()

In [8]:
#экспорт файла2
cascconstant=pd.read_csv('D:\CASC_Constant.csv', sep=',')
cascconstant.head()

,CustomerId,ActivationDate,Age,Sex,SubscribedEmail,SubscribedPush
0,2728183,2015-01-01,21.0,Female,False,True
1,2728198,2015-01-01,21.0,Female,True,True
2,2728306,2015-01-01,21.0,Female,True,True
3,2728178,2015-01-01,22.0,Male,True,True
4,2728322,2015-01-01,22.0,Male,True,True


In [18]:
cascconstant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8856 entries, 0 to 9998
Data columns (total 6 columns):
CustomerId         8856 non-null int64
ActivationDate     8856 non-null object
Age                8856 non-null float64
Sex                8856 non-null object
SubscribedEmail    8856 non-null bool
SubscribedPush     8856 non-null bool
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 363.2+ KB


In [ ]:
#преобразование данных к типу дата
cascconstant['RKDate'] = pd.to_datetime(cascconstant['RKDate'])
cascconstant.info()

In [10]:
#проверка и очистка выборки стррок с пустыми значениями файла1
print (cascresto.shape)
cascresto = cascresto.dropna() 

(881608, 9)


In [11]:
print (cascresto.shape)

(881608, 9)


In [12]:
#проверка и очистка выборки стррок с пустыми значениями файла2
print (cascconstant.shape)
cascconstant = cascconstant.dropna() 

(10000, 6)


In [13]:
print (cascconstant.shape)

(8856, 6)


In [17]:
startdate=datetime.datetime(2017,7,1)
enddate=datetime.datetime(2017,12,31)
#поготовка данных
group=cascresto.groupby(['CustomerID'])
def build_statistics(record):
    return pd.Series({    
        # флаг = зависимая переменная Y
        'Flag': len(record.loc[(startdate <= record['RKDate']) & (enddate >= record['RKDate'])]) > 0,
        # Количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01
        'Recency': pd.Timedelta(startdate - record[startdate > record['RKDate']]['RKDate'].max()).days,
        # Средняя частота походов клиентов на дату 2017-07-01, измеряется как среднее количество визитов в месяц за время между 2017-07-01 и первым визитом клиента
        'Frequency': record[record['RKDate'] < startdate].groupby([record[record['RKDate'] < startdate]['RKDate'].dt.year, record[record['RKDate'] < startdate]['RKDate'].dt.month]).size().mean(),
        # Средний чек клиента по всем покупкам до 2017-07-01
        'Monetary_value': record[startdate > record['RKDate']]['SummAfterPointsUsage'].mean(),
        # Сколько раз использовалась скидка
        'Times_using_discount': len(record[record['SummBasic']!=record['SummAfterPointsUsage']])
    })

statistics = group.apply(build_statistics)
statistics.head()

,Flag,Recency,Frequency,Monetary_value,Times_using_discount
CustomerID,,,,,
2728046,False,160.0,8.142857,251.789474,3
2728088,False,720.0,17.000000,279.647059,0
2728089,True,15.0,6.600000,398.734848,16
2728095,False,177.0,9.000000,262.737374,42
2728107,True,115.0,5.250000,393.714286,7


In [26]:
#соеденить 2 файла
fulltable = pd.merge(left=statistics, right=cascconstant[['CustomerId','Sex', 'Age']], left_on='CustomerID', right_on='CustomerId', how='right')
fulltable=fulltable.dropna()
fulltable.head()

,Flag,Recency,Frequency,Monetary_value,Times_using_discount,CustomerId,Sex,Age
0,False,160.0,8.142857,251.789474,3,2728046,Female,24.0
1,False,720.0,17.000000,279.647059,0,2728088,Male,46.0
2,True,15.0,6.600000,398.734848,16,2728089,Female,27.0
3,False,177.0,9.000000,262.737374,42,2728095,Female,54.0
4,True,115.0,5.250000,393.714286,7,2728107,Male,48.0


In [27]:
#Разбеение клиентов случайно на две группы в соотношении 80:20 на тестовую и обучающую выборку
train, test = train_test_split(fulltable, test_size=0.2)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1760 entries, 8742 to 7005
Data columns (total 8 columns):
Flag                    1760 non-null bool
Recency                 1760 non-null float64
Frequency               1760 non-null float64
Monetary_value          1760 non-null float64
Times_using_discount    1760 non-null int64
CustomerId              1760 non-null int64
Sex                     1760 non-null object
Age                     1760 non-null float64
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 111.7+ KB


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7040 entries, 6814 to 6022
Data columns (total 8 columns):
Flag                    7040 non-null bool
Recency                 7040 non-null float64
Frequency               7040 non-null float64
Monetary_value          7040 non-null float64
Times_using_discount    7040 non-null int64
CustomerId              7040 non-null int64
Sex                     7040 non-null object
Age                     7040 non-null float64
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 446.9+ KB


In [55]:
X_train = train[['Recency', 'Frequency', 'Monetary_value']]
Y_train = train.Flag 
X_test = test[['Recency', 'Frequency', 'Monetary_value']]
Y_test = test.Flag 

In [56]:
classifier = LogisticRegression(solver='lbfgs',random_state=0)

In [57]:
classifier.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
Y_pred = classifier.predict(X_test)

In [61]:
Y_pred

array([ True, False,  True, ...,  True, False,  True])

In [66]:
print('Score: ' + str(classifier.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7295454545454545
Precision: 0.818955042527339
Recall: 0.6733266733266733
